In [1]:
from get_training_data import *

import tensorflow as tf
import keras
import pickle
import gzip
import os
import random
import matplotlib.pyplot as plt
%matplotlib nbagg

/home/andy/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with gzip.open('train_cases.pkl.gz','rb') as fp:
    train_cases = pickle.load(fp)
type(train_cases), len(train_cases)

(tuple, 2)

In [3]:
train_X = train_cases[0]
train_y = train_cases[1]
ntrain = train_X.shape[0]
type(train_X), train_X.shape, train_X.dtype, type(train_y), train_y.shape, train_y.dtype

(numpy.ndarray,
 (16384, 16, 16, 128),
 dtype('float32'),
 numpy.ndarray,
 (16384, 1),
 dtype('float64'))

In [4]:
with gzip.open('valid_cases.pkl.gz','rb') as fp:
    valid_cases = pickle.load(fp)
valid_X = valid_cases[0]
valid_y = valid_cases[1]
nvalid = valid_X.shape[0]
type(valid_X), valid_X.shape, valid_X.dtype, type(valid_y), valid_y.shape, valid_y.dtype

(numpy.ndarray,
 (4096, 16, 16, 128),
 dtype('float32'),
 numpy.ndarray,
 (4096, 1),
 dtype('float64'))

In [5]:
def choose_training_case():
    i = random.randint(0, ntrain-1)
    return(train_X[i,:,:,:], train_y[i,:])

def choose_validation_case():
    i = random.randint(0, nvalid-1)
    return(valid_X[i,:,:,:], valid_y[i,:])

In [6]:
ADD_CONST = 1e-5
CENTER = -2.5
SCALE = 5
XFORM = True

In [7]:
def xform(a):
    if XFORM:
        return((np.log(a+ADD_CONST) - CENTER) / SCALE)
    else:
        return(a)

In [8]:
def make_batch_from_megabatch(batchsize=32, validation=False):
    get_case = choose_validation_case if validation else choose_training_case    
    xbatch = np.empty([0, INPUTS_PER_BEAT, NBEATS, NCHANNELS],dtype=np.float32)
    ybatch = np.empty([0, 1])
    for i in range(batchsize):
        case = get_case()
        x = np.expand_dims(case[0], axis=0)
        xbatch = np.concatenate([xbatch, x], axis=0)
        y = np.array(int(case[1])).reshape([1,1])
        ybatch = np.concatenate([ybatch, y], axis=0)
    return(xform(xbatch), ybatch)

def train_gen(batchsize=32):
    while True:
        yield(make_batch_from_megabatch(batchsize, validation=False))

def valid_gen(batchsize=32):
    while True:
        yield(make_batch_from_megabatch(batchsize, validation=True))

In [9]:
# The generators are way too slow: have to fix this
for t in train_gen():
    break
t[0].shape, t[1].shape

((32, 16, 16, 128), (32, 1))

In [10]:
for v in valid_gen():
    break
v[0].shape, v[1].shape

((32, 16, 16, 128), (32, 1))

In [606]:
input_shape = (INPUTS_PER_BEAT, NBEATS, NCHANNELS)

model = keras.Sequential()
model.add(keras.layers.Conv2D(64, 1, input_shape=input_shape)) # 64
#model.add(keras.layers.Conv2D(80, 1, input_shape=input_shape))
#model.add(keras.layers.PReLU())
model.add(keras.layers.LeakyReLU(0))
#model.add(keras.layers.MaxPooling2D(2,1))
#model.add(keras.layers.BatchNormalization())
model.add(keras.layers.SpatialDropout2D(.1))
model.add(keras.layers.Conv2D(32, 1))  # 32
#model.add(keras.layers.Conv2D(50, 1))  # 32
#model.add(keras.layers.PReLU())
model.add(keras.layers.LeakyReLU(0))
model.add(keras.layers.BatchNormalization())
#model.add(keras.layers.SpatialDropout2D(.15))
#model.add(keras.layers.Conv2D(16, 1, activation='relu'))
#model.add(keras.layers.BatchNormalization())
#model.add(keras.layers.Conv2D(12, (2,4))) # 16
model.add(keras.layers.Conv2D(12, (1,4))) # 16
#model.add(keras.layers.Conv2D(16, (2,4))) # 16
#model.add(keras.layers.PReLU())
model.add(keras.layers.LeakyReLU(0.0))
#model.add(keras.layers.BatchNormalization())
model.add(keras.layers.SpatialDropout2D(.3))
#model.add(keras.layers.Conv2D(25, (4,5))) # 29,5
model.add(keras.layers.Conv2D(25, (4,6))) # 29,5
#model.add(keras.layers.Conv2D(32, (4,6))) # 29,5
#model.add(keras.layers.Conv2D(25, 5)) # 29,5
#model.add(keras.layers.PReLU())
model.add(keras.layers.LeakyReLU(0.0))
model.add(keras.layers.BatchNormalization())
#model.add(keras.layers.SpatialDropout2D(.25))
#model.add(keras.layers.Conv2D(3, (2,5))) # 4,3
#model.add(keras.layers.Conv2D(4, (2,4))) # 4,3
#model.add(keras.layers.Conv2D(3, (2,4))) # 4,3
model.add(keras.layers.Conv2D(3, (1,4))) # 4,3
#model.add(keras.layers.Conv2D(4, 3)) # 4,3
model.add(keras.layers.LeakyReLU(0.0))
#model.add(keras.layers.MaxPooling2D(1,2))
#model.add(keras.layers.PReLU())
#model.add(keras.layers.BatchNormalization())
#model.add(keras.layers.Conv2D(3, 1))
#model.add(keras.layers.LeakyReLU(0))
model.add(keras.layers.Flatten())
#model.add(keras.layers.Dropout(.2)) # or .2
#model.add(keras.layers.Dropout(.25)) # or .2
#model.add(keras.layers.Dense(35, activation='relu'))
#model.add(keras.layers.Dense(25, activation='relu'))
model.add(keras.layers.Dropout(.6)) # .5 or .6
#model.add(keras.layers.GaussianDropout(.5))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [607]:
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(lr=.003,decay=2e-4), 
              metrics=['accuracy'])

In [608]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_596 (Conv2D)          (None, 16, 16, 64)        8256      
_________________________________________________________________
leaky_re_lu_596 (LeakyReLU)  (None, 16, 16, 64)        0         
_________________________________________________________________
spatial_dropout2d_239 (Spati (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_597 (Conv2D)          (None, 16, 16, 32)        2080      
_________________________________________________________________
leaky_re_lu_597 (LeakyReLU)  (None, 16, 16, 32)        0         
_________________________________________________________________
batch_normalization_239 (Bat (None, 16, 16, 32)        128       
_________________________________________________________________
conv2d_598 (Conv2D)          (None, 16, 13, 12)        1548      
__________

In [609]:
valid=valid_gen()
train=train_gen()
model.fit_generator(train, 
                    validation_data=valid, validation_steps=64,
                    epochs=60, 
                    steps_per_epoch=64)

Epoch 1/60
64/64 [==============================] - 12s 184ms/step - loss: 0.6680 - acc: 0.7129 - val_loss: 0.6105 - val_acc: 0.7573
Epoch 2/60
64/64 [==============================] - 2s 24ms/step - loss: 0.5965 - acc: 0.7573 - val_loss: 0.7006 - val_acc: 0.7441
Epoch 3/60
64/64 [==============================] - 2s 24ms/step - loss: 0.5724 - acc: 0.7739 - val_loss: 0.6291 - val_acc: 0.7622
Epoch 4/60
64/64 [==============================] - 2s 24ms/step - loss: 0.5557 - acc: 0.7773 - val_loss: 0.5790 - val_acc: 0.7573
Epoch 5/60
64/64 [==============================] - 2s 24ms/step - loss: 0.5396 - acc: 0.7769 - val_loss: 0.5483 - val_acc: 0.7583
Epoch 6/60
64/64 [==============================] - 2s 24ms/step - loss: 0.5357 - acc: 0.7612 - val_loss: 0.5211 - val_acc: 0.7695
Epoch 7/60
64/64 [==============================] - 2s 24ms/step - loss: 0.5267 - acc: 0.7578 - val_loss: 0.5157 - val_acc: 0.7500
Epoch 8/60
64/64 [==============================] - 2s 24ms/step - loss: 0.5009 -

In [610]:
for i in range(10):
    song, tempo, compatible, clip = get_validation_case()
    print( song, tempo )
    c = clip_to_tf_input(resample_clip(clip))
    x = np.expand_dims(xform(c), axis=0)
    p = model.predict(x)[0][0]
    print( 'Predicted: ', p, '   Actual: ', compatible, '\n' )

GoodGollyMissMollyWithJerryLeeAndFats 287
Predicted:  0.011747412    Actual:  False 

ItsRainingMen 55
Predicted:  3.1052994e-11    Actual:  False 

53rdAnd3rdMP 136
Predicted:  0.8852352    Actual:  True 

53rdAnd3rdMP 46
Predicted:  2.7495596e-05    Actual:  False 

SmoothOperator 48
Predicted:  0.00070420053    Actual:  False 

WalkThisWay1 43
Predicted:  2.4785813e-06    Actual:  False 

Kalamazoo 99
Predicted:  0.0045318445    Actual:  False 

IllBeThereForYou 43
Predicted:  0.00271275    Actual:  False 

ChatanoogaChooChoo 58
Predicted:  0.14735419    Actual:  False 

WalkThisWay1 73
Predicted:  0.00036416837    Actual:  False 



<code>
(12,1x4) -> 2282, 2250, 2259, *2247*, **2233**, 2155, 2174, 2493, *2228*, 2386, 2060,
            2049, 2008
(12,2x3) -> 2083, *2222*, 2807, 2188, **2266**, 2498, *2375*
(12,2x5) -> 2297
(25,4x5) -> 2170, *2259*, *2357*, **2312**, 2237, 2830, 2228, 2378, 2366
(25,4x7) -> 2537, 2282
(25,5x6) -> **2372**, *2234*, 2225, 2160, 2567, *2410*, 2482
(25,3x6) -> *2259*, 2257, 2361, 2222, 2068, 2212, 2415, 2416, *2305*, 2253, **2262**,
            2410, 2487
    
New baseline from below:
(3,1x4) -> *2263*, 2097, 2158, 2106, 2357, 2242, *2248*, 2284, 2484, **2256**, 2572,
           2243, 2244, 2359, 2303, 2371, 2247, 2382, 2481, 2236, 2108</code>

<code>
(3,1x4) -> *2263*, 2097, 2158, 2106, 2357, 2242, *2248*, 2284, 2484, **2256**, 2572,
           2243, 2244, 2359, 2303, 2371, 2247, 2382, 2481, 2236, 2108
           
(3,3x4) -> 2277, 2867
(3,2x3) -> 2232, 2254, 2247, 2303, 2356, 2396, 2400, 2193, 2385, 2200, 2084, 
           *2275*, 2541, 2411, 2539, 2124, 2250, 2371, 2119, **2279**, 2188,
           2289, *2280*, 
           
80/50/12/25/3 f -> 2384, 2366, 2168, 2139, *2280*, 2440, **2285**, *2267*, 2429
50/20/12/25/3 f -> 2164, 2065, 2497, 2370, 2405, **2381**, 2393
15/30/60 d/o -> 2361, 2205, 2083, 2267, 2424, 2342, 2214, 2340, 2143, 2145, **2302**,
                2311, *2293*, 2348, *2336*
    
New baseline from below:
*16f to 12f -> 2122, 2283, 2089, 2319, 2251, 2427, 2291, 2353, 2272, 2410, *2276*,
               2309, 2161, 2349, 2260, 2229, 2250, 2470, **2275**, 2334, 2266,
               *2269*, 2088*
</code>

<code>
*16f to 12f -> 2122, *2283*, 2089, 2319, 2251, 2427, 2291, 2353, *2272*, 2410, **2276**,
               2309, 2161, 2349, 2260, 2229, 2250 *
    
New baseline from below:
10//30//60 -> 2284, 2156, 2438, 2451, 2139, 2457, 2281, 2430, 2454, 2328, 2610, 
              **2295**, 2403, 2095, 2597, 2358, 2155, 2241, *2302*, 2377, 2132,
              2290, 2586, 2235, *2292*, 2225, 2213, 2335, 2291
</code>

<code>
below w/o 10 -> 2455, 2336
*10//30//60 w 4f on top -> 2306, 2510, 2263, 2279, *2319*, 2163, 2368, *2337*, 2396, 
                           2167, 2138, 2238, 3505, 2114, 2465, 2359, **2329**, 2581, 2177,
                           2415, 2517 *
[above w extra b/n bottom -> 2646, 2336]
10//30//60 -> 2284, 2156, 2438, 2451, 2139, 2457, 2281, 2430, 2454, 2328, 2610, 
              **2295**, 2403, 2095, 2597, 2358, 2155, 2241, *2302*, 2377, 2132,
              2290, 2586, 2235, *2292*, 2225, 2213
/15//25/55 -> 2461, 2321, 2453, 2737
//40//50   -> 2555
//20/30/60 -> 2789
/20//30/60 -> *2394*, 2196, 2450, **2368**, *2359*, 2541, 2327
//40//60   -> 2227, *2368*, 2170, 2458, 2274, *2429*, **2419**, 2463, 2497
//30//70   -> 2442, 2358
//30//50   -> 2509, 2552
    
baseline: preproc sp d/o replaces b/n for middle layer (d/o=//30//60) -> 
    2258, 2308, 2236, 2585, 2487, 2599, 2238, *2326*, 2316, 2395, 2484,
    **2335**, 2280, *2337*, 2693, 2272, 2523, 2532, 2305, 2562, 2222
</code>    